In [1]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

import numpy as np
import pandas as pd

from keras.utils import np_utils
import itertools

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score

In [2]:
data = np.load('/dbfs/FileStore/tables/ORL_faces.npz') 

x_train = data['trainX']
#normalize every image
x_train = np.array(x_train,dtype='float32')/255

x_test = data['testX']
x_test = np.array(x_test,dtype='float32')/255

# load the Label of Images
y_train= data['trainY']
y_test= data['testY']

# show the train and test Data format
print('x_train : {}'.format(x_train[:]))
print('Y-train shape: {}'.format(y_train))
print('x_test shape: {}'.format(x_test.shape))

x_train : [[0.1882353 0.19215687 0.1764706 ... 0.18431373 0.18039216 0.18039216]
 [0.23529412 0.23529412 0.24313726 ... 0.1254902 0.13333334 0.13333334]
 [0.15294118 0.17254902 0.20784314 ... 0.11372549 0.10196079 0.11372549]
 ...
 [0.44705883 0.45882353 0.44705883 ... 0.38431373 0.3764706 0.38431373]
 [0.4117647 0.4117647 0.41960785 ... 0.21176471 0.18431373 0.16078432]
 [0.45490196 0.44705883 0.45882353 ... 0.37254903 0.39215687 0.39607844]]
Y-train shape: [ 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3
 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5
 6 6 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7 7 7 7 7 7 7 7
 8 8 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 9 9 9
 10 10 10 10 10 10 10 10 10 10 10 10 11 11 11 11 11 11 11 11 11 11 11 11
 12 12 12 12 12 12 12 12 12 12 12 12 13 13 13 13 13 13 13 13 13 13 13 13
 14 14 14 14 14 14 14 14 14 14 14 14 15 15 15 15 15 15 15 15 15 15 15 15
 16 16 16 16 16 16 16 16 16 16 16 16 17 17 17 17 17 17 17 17 17 17 17 17
 18 18 18 18 18 18 18 18 18 18 18 18 19 19 19 19 19 19 19 19 19 19 19 19]
x_test shape: (160, 10304)

In [3]:
x_train, x_valid, y_train, y_valid= train_test_split(
    x_train, y_train, test_size=.05, random_state=1234,)

In [4]:
im_rows=112
im_cols=92
batch_size=512
im_shape=(im_rows, im_cols, 1)

#change the size of images
x_train = x_train.reshape(x_train.shape[0], *im_shape)
x_test = x_test.reshape(x_test.shape[0], *im_shape)
x_valid = x_valid.reshape(x_valid.shape[0], *im_shape)

print('x_train shape: {}'.format(y_train.shape[0]))
print('x_test shape: {}'.format(y_test.shape))

x_train shape: 228
x_test shape: (160,)

In [5]:
cnn_model= Sequential([
    Conv2D(filters=36, kernel_size=7, activation='relu', input_shape= im_shape),
    MaxPooling2D(pool_size=2),
    Conv2D(filters=54, kernel_size=5, activation='relu', input_shape= im_shape),
    MaxPooling2D(pool_size=2),
    Flatten(),
    Dense(2024, activation='relu'),
     Dropout(0.5),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    #20 is the number of outputs
    Dense(20, activation='softmax')  
])

cnn_model.compile(
    loss='sparse_categorical_crossentropy',#'categorical_crossentropy',
    optimizer=Adam(lr=0.0001),
    metrics=['accuracy']
)

In [6]:
cnn_model.summary()

_________________________________________________________________
Layer (type) Output Shape Param # 
=================================================================
conv2d_1 (Conv2D) (None, 106, 86, 36) 1800 
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 53, 43, 36) 0 
_________________________________________________________________
conv2d_2 (Conv2D) (None, 49, 39, 54) 48654 
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 24, 19, 54) 0 
_________________________________________________________________
flatten_1 (Flatten) (None, 24624) 0 
_________________________________________________________________
dense_1 (Dense) (None, 2024) 49841000 
_________________________________________________________________
dropout_1 (Dropout) (None, 2024) 0 
_________________________________________________________________
dense_2 (Dense) (None, 1024) 2073600 
_________________________________________________________________
dropout_2 (Dropout) (None, 1024) 0 
_________________________________________________________________
dense_3 (Dense) (None, 512) 524800 
_________________________________________________________________
dropout_3 (Dropout) (None, 512) 0 
_________________________________________________________________
dense_4 (Dense) (None, 20) 10260 
=================================================================
Total params: 52,500,114
Trainable params: 52,500,114
Non-trainable params: 0
_________________________________________________________________

In [7]:
history=cnn_model.fit(
    np.array(x_train), np.array(y_train), batch_size=512,
    epochs=250, verbose=2,
    validation_data=(np.array(x_valid),np.array(y_valid)),
)

Train on 228 samples, validate on 12 samples
Epoch 1/250
 - 6s - loss: 3.0150 - acc: 0.0526 - val_loss: 3.0079 - val_acc: 0.0833
Epoch 2/250
 - 3s - loss: 2.9981 - acc: 0.0526 - val_loss: 3.0233 - val_acc: 0.0833
Epoch 3/250
 - 3s - loss: 3.0197 - acc: 0.0482 - val_loss: 3.0230 - val_acc: 0.0000e+00
Epoch 4/250
 - 3s - loss: 3.0064 - acc: 0.0746 - val_loss: 3.0320 - val_acc: 0.0000e+00
Epoch 5/250
 - 3s - loss: 2.9687 - acc: 0.0965 - val_loss: 3.0324 - val_acc: 0.0000e+00
Epoch 6/250
 - 4s - loss: 2.9784 - acc: 0.0789 - val_loss: 3.0279 - val_acc: 0.0000e+00
Epoch 7/250
 - 3s - loss: 2.9924 - acc: 0.0658 - val_loss: 3.0228 - val_acc: 0.0000e+00
Epoch 8/250
 - 4s - loss: 2.9842 - acc: 0.0658 - val_loss: 3.0178 - val_acc: 0.0000e+00
Epoch 9/250
 - 3s - loss: 2.9839 - acc: 0.0833 - val_loss: 3.0161 - val_acc: 0.0000e+00
Epoch 10/250
 - 4s - loss: 2.9664 - acc: 0.0614 - val_loss: 3.0132 - val_acc: 0.0000e+00
Epoch 11/250
 - 3s - loss: 2.9650 - acc: 0.0526 - val_loss: 3.0117 - val_acc: 0.0000e+00
Epoch 12/250
 - 4s - loss: 2.9652 - acc: 0.0921 - val_loss: 3.0102 - val_acc: 0.0000e+00
Epoch 13/250
 - 4s - loss: 2.9718 - acc: 0.0658 - val_loss: 3.0087 - val_acc: 0.0000e+00
Epoch 14/250
 - 6s - loss: 2.9691 - acc: 0.0833 - val_loss: 3.0037 - val_acc: 0.0000e+00
Epoch 15/250
 - 6s - loss: 2.9480 - acc: 0.0965 - val_loss: 2.9977 - val_acc: 0.0000e+00
Epoch 16/250
 - 4s - loss: 2.9371 - acc: 0.1096 - val_loss: 2.9924 - val_acc: 0.0000e+00
Epoch 17/250
 - 4s - loss: 2.9149 - acc: 0.1272 - val_loss: 2.9866 - val_acc: 0.0000e+00
Epoch 18/250
 - 3s - loss: 2.9315 - acc: 0.1316 - val_loss: 2.9820 - val_acc: 0.0000e+00
Epoch 19/250
 - 3s - loss: 2.9256 - acc: 0.1228 - val_loss: 2.9788 - val_acc: 0.0000e+00
Epoch 20/250
 - 3s - loss: 2.9088 - acc: 0.1535 - val_loss: 2.9761 - val_acc: 0.0000e+00
Epoch 21/250
 - 4s - loss: 2.8989 - acc: 0.1447 - val_loss: 2.9737 - val_acc: 0.0000e+00
Epoch 22/250
 - 3s - loss: 2.8860 - acc: 0.1711 - val_loss: 2.9704 - val_acc: 0.0000e+00
Epoch 23/250
 - 4s - loss: 2.8786 - acc: 0.1623 - val_loss: 2.9652 - val_acc: 0.0000e+00
Epoch 24/250
 - 3s - loss: 2.8832 - acc: 0.1272 - val_loss: 2.9575 - val_acc: 0.0000e+00
Epoch 25/250
 - 4s - loss: 2.8693 - acc: 0.1447 - val_loss: 2.9473 - val_acc: 0.0000e+00
Epoch 26/250
 - 3s - loss: 2.8528 - acc: 0.1754 - val_loss: 2.9330 - val_acc: 0.0000e+00
Epoch 27/250
 - 3s - loss: 2.7948 - acc: 0.2281 - val_loss: 2.9167 - val_acc: 0.0000e+00
Epoch 28/250
 - 3s - loss: 2.7801 - acc: 0.1711 - val_loss: 2.8979 - val_acc: 0.0833
Epoch 29/250
 - 3s - loss: 2.7693 - acc: 0.2544 - val_loss: 2.8764 - val_acc: 0.0833
Epoch 30/250
 - 4s - loss: 2.7891 - acc: 0.1535 - val_loss: 2.8479 - val_acc: 0.0833
Epoch 31/250
 - 4s - loss: 2.7271 - acc: 0.2237 - val_loss: 2.8234 - val_acc: 0.0833
Epoch 32/250
 - 4s - loss: 2.7560 - acc: 0.2018 - val_loss: 2.7969 - val_acc: 0.0833
Epoch 33/250
 - 3s - loss: 2.7061 - acc: 0.2544 - val_loss: 2.7663 - val_acc: 0.1667
Epoch 34/250
 - 3s - loss: 2.6489 - acc: 0.2763 - val_loss: 2.7314 - val_acc: 0.1667
Epoch 35/250
 - 3s - loss: 2.6140 - acc: 0.2719 - val_loss: 2.6905 - val_acc: 0.1667
Epoch 36/250
 - 4s - loss: 2.6357 - acc: 0.2632 - val_loss: 2.6480 - val_acc: 0.1667
Epoch 37/250
 - 3s - loss: 2.5711 - acc: 0.2763 - val_loss: 2.6067 - val_acc: 0.2500
Epoch 38/250
 - 3s - loss: 2.5515 - acc: 0.2719 - val_loss: 2.5660 - val_acc: 0.2500
Epoch 39/250
 - 3s - loss: 2.5266 - acc: 0.3158 - val_loss: 2.5251 - val_acc: 0.2500
Epoch 40/250
 - 3s - loss: 2.3940 - acc: 0.3904 - val_loss: 2.4771 - val_acc: 0.4167
Epoch 41/250
 - 5s - loss: 2.3730 - acc: 0.3596 - val_loss: 2.4303 - val_acc: 0.4167
Epoch 42/250
 - 3s - loss: 2.3219 - acc: 0.3816 - val_loss: 2.3866 - val_acc: 0.3333
Epoch 43/250
 - 3s - loss: 2.2418 - acc: 0.3991 - val_loss: 2.3310 - val_acc: 0.2500
Epoch 44/250
 - 3s - loss: 2.2465 - acc: 0.3684 - val_loss: 2.2714 - val_acc: 0.3333
Epoch 45/250
 - 3s - loss: 2.2170 - acc: 0.3465 - val_loss: 2.2120 - val_acc: 0.3333
Epoch 46/250
 - 4s - loss: 2.1561 - acc

In [8]:
scor = cnn_model.evaluate( np.array(x_test),  np.array(y_test), verbose=0)

print('test los {:.4f}'.format(scor[0]))
print('test acc {:.4f}'.format(scor[1]))

test los 0.3462
test acc 0.9375